In [1]:
# Imports iniciais
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import joblib
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
import ipywidgets as widgets
from IPython.display import display, clear_output
import itertools

# === CARREGAR MODELO TREINADO ===
modelo = joblib.load('modelo_4_camadas_ESRS_ESRD_COMPLETO.pkl')
weights = modelo['weights']
scaler = modelo['scaler']
aeronaves_dict = modelo['aeronaves']

# === FUNÇÕES RNA ===
def tanh(x): return np.tanh(x)
def neural_network(weights, X):
    W1 = weights[:9 * 10].reshape((9, 10))
    b1 = weights[90:100]
    W2 = weights[100:180].reshape((10, 8))
    b2 = weights[180:188]
    W3 = weights[-9:-1].reshape((8, 1))
    b3 = weights[-1]
    hidden1 = tanh(np.dot(X, W1) + b1)
    hidden2 = tanh(np.dot(hidden1, W2) + b2)
    output = np.dot(hidden2, W3) + b3
    return output.flatten()

def prever_deformacao_customizada(valores):
    entrada = np.array([valores])
    entrada_norm = scaler.transform(entrada)
    return neural_network(weights, entrada_norm)[0]

# === AUSTROADS: Temperatura média anual ===
def calcular_temperaturas(temp_media_anual):
    return None, temp_media_anual

def corrigir_mr_para_Tc(E25_mpa, temp_media_anual):
    return E25_mpa * np.exp(-0.08 * (temp_media_anual - 25))

# === INTERFACE PARA CONFIABILIDADE ===
covs_padrao = {
    'revest_MR': 15, 'base_MR': 20, 'subbase_MR': 20, 'subleito_MR': 20,
    'revest_h': 7, 'base_h': 12, 'subbase_h': 15
}

cov_widgets = {
    key: widgets.FloatText(value=val, description=f'COV {key} (%)')
    for key, val in covs_padrao.items()
}

nivel_confianca = widgets.FloatText(value=95.0, description='Confiabilidade (%)')

confiab_section = widgets.VBox([
    widgets.HTML('<b>Coeficientes de Variação (COV):</b>')
] + list(cov_widgets.values()) + [nivel_confianca])

def calcular_confiabilidade_rosenblueth(aeronave, media_inputs):
    indices = {
        'revest_MR': 2, 'revest_h': 3,
        'base_MR': 4, 'base_h': 5,
        'subbase_MR': 6, 'subbase_h': 7,
        'subleito_MR': 8
    }
    variaveis = list(indices.items())
    mu_vector = media_inputs.copy()
    cov_values = [cov_widgets[k].value / 100 for k, _ in variaveis]

    # Verificação se todos os COVs são zero
    if all(cov == 0 for cov in cov_values):
        print("⚠️ Atenção: Todos os COVs estão zerados. A análise será determinística (sem incertezas).")

    # Geração de todas as combinações +1/-1 (total 2^7 = 128)
    combinacoes = list(itertools.product([-1, 1], repeat=7))
    resultados = []
    for sinais in combinacoes:
        v = mu_vector.copy()
        for i, (_, idx) in enumerate(variaveis):
            v[idx] = mu_vector[idx] * (1 + sinais[i] * cov_values[i])
        f = prever_deformacao_customizada(v)
        resultados.append(f)

    media = np.mean(resultados)
    desvio = np.std(resultados)
    z = stats.norm.ppf(nivel_confianca.value / 100)
    confiavel = media + z * desvio
    return media, confiavel

# === CÁLCULO DE COBERTURAS ===
def calcular_C(ev):
    if ev >= 1.765e-3:
        return (0.00414131 / ev) ** 8.1
    else:
        log_C = (-0.1638 + 185.19 * ev) ** (-0.60586)
        return 10 ** log_C

def calcular_N(a, b, L):
    return (1 + (b * L / 200)) * a * L

def calcular_cp(faixa_centro, weq, pos_pneus, wander_std):
    return sum([
        (stats.norm.cdf(faixa_centro + weq/2, pneu_pos, wander_std) -
         stats.norm.cdf(faixa_centro - weq/2, pneu_pos, wander_std))
        for pneu_pos in pos_pneus
    ])

def obter_pc_por_faixa(aeronave_nome, h_total, dados_aeronaves, wander_std=77.3):
    faixa_largura = 25.4
    num_faixas = 81
    faixas = np.linspace(-40 * faixa_largura, 40 * faixa_largura, num_faixas)
    for aero in dados_aeronaves:
        if aero['nome'] == aeronave_nome:
            if aero['Ne'] == 1 or h_total >= aero['t'] - aero['w']:
                weq = aero['w'] + aero['t'] + h_total if aero['Ne'] > 1 else aero['w'] + h_total
                posicoes_pneus = [aero['xk_centro']]
            else:
                weq = aero['w'] + h_total
                deslocamento = aero['t'] / 2
                posicoes_pneus = [aero['xk_centro'] - deslocamento, aero['xk_centro'] + deslocamento]
            posicoes_pneus_simetrico = posicoes_pneus + [-pos for pos in posicoes_pneus]
            cp_values = np.array([calcular_cp(f, weq, posicoes_pneus_simetrico, wander_std) for f in faixas])
            pc_values = 1 / cp_values
            return faixas, pc_values
    raise ValueError(f"Aeronave {aeronave_nome} não encontrada.")

# === DADOS DAS AERONAVES ===
dados_aeronaves_completos = [
    {'nome': 'A-320',   'w': 31.5,   't': 92.69, 'xk_centro': 333.15, 'Ne': 2},
    {'nome': 'A-321',   'w': 34.7, 't': 92.71, 'xk_centro': 379.5,  'Ne': 2},
    {'nome': 'B-737',   'w': 32.3, 't': 86.36, 'xk_centro': 285.75, 'Ne': 2},
    {'nome': 'EMB-195', 'w': 29.2, 't': 86.36, 'xk_centro': 297.18, 'Ne': 2},
    {'nome': 'ATR-72',  'w': 22.6, 't': 43.60, 'xk_centro': 205.0,  'Ne': 2},
    {'nome': 'CESSNA',  'w': 16.9, 't': 0, 'xk_centro': 208.28,  'Ne': 1},
]

# === INTERFACE ===
temp_anual = widgets.FloatText(description='Temp. média anual (°C):', value=25.0)
revest_E = widgets.FloatText(description='MR 25°C (MPa) Revestimento:', value=2500)
revest_h = widgets.FloatText(description='h revest. (m):', value=0.3)
base_E = widgets.FloatText(description='MR base (MPa):', value=300)
base_h = widgets.FloatText(description='h base (m):', value=0.3)
subbase_E = widgets.FloatText(description='MR subbase (MPa):', value=150)
subbase_h = widgets.FloatText(description='h subbase (m):', value=0.4)
subleito_E = widgets.FloatText(description='MR subleito (MPa):', value=60)
vida_util = widgets.FloatText(description='Vida útil (anos):', value=20)

# === Wander std com explicação ===
wander_std_explica = widgets.HTML(
    "<i>O valor padrão do desvio-padrão do wander é 77,3 cm, conforme recomendação da FAA. "
    "Altere se desejar simular outros cenários de distribuição lateral.</i>"
)
wander_std_widget = widgets.FloatText(description='Wander std (cm):', value=77.3)

# === Explicação sobre COV ===
cov_explica = widgets.HTML(
    "<i>Os Coeficientes de Variação (COV) representam o grau de incerteza associado a cada variável de entrada. "
    "Eles são expressos em porcentagem e indicam o quanto um parâmetro pode variar em torno de seu valor médio. "
    "Se desejar uma análise determinística (sem incertezas), defina todos os COVs como zero. Os COVs dispostos foram de acordo com a literatura</i>"
)
# === CAMPOS PARA AERONAVES ===
aeronave_inputs = []
for nome in sorted(aeronaves_dict.keys()):
    dec = widgets.IntText(description=f'{nome} - a:', value=0, layout=widgets.Layout(width='200px'))
    cres = widgets.FloatText(description='b (%):', value=0.0, layout=widgets.Layout(width='150px'))
    aeronave_inputs.append((nome, dec, cres))

# === BOTÃO E SAÍDA ===
executar = widgets.Button(description="Gerar gráfico CDF acumulado", button_style='success')
saida = widgets.Output()

def rodar_plot(b):
    with saida:
        clear_output()
        faixas = np.linspace(-40 * 25.4, 40 * 25.4, 81)
        cdf_total = np.zeros_like(faixas)
        plt.figure(figsize=(12, 6))

        _, Tc = calcular_temperaturas(temp_anual.value)
        revest_E_corrigido = corrigir_mr_para_Tc(revest_E.value, Tc)
        h_total_cm = (revest_h.value + base_h.value + subbase_h.value) * 100
        fx_ref = None

        for nome, dec, cres in aeronave_inputs:
            if dec.value > 0:
                entrada_original = [
                    aeronaves_dict[nome]['Pressão dos Pneus(MPa)'],
                    aeronaves_dict[nome]['dist_rodas(m)'],
                    revest_E.value, revest_h.value,
                    base_E.value, base_h.value,
                    subbase_E.value, subbase_h.value,
                    subleito_E.value
                ]

                entrada_corrigida = entrada_original.copy()
                entrada_corrigida[2] = revest_E_corrigido

                ev_original = prever_deformacao_customizada(entrada_original)
                ev_corrigida = prever_deformacao_customizada(entrada_corrigida)
                ev_media, ev_confiavel = calcular_confiabilidade_rosenblueth(nome, entrada_corrigida)

                print(f"Aeronave: {nome}")
                print(f" - MR original: {revest_E.value:.2f} MPa | Deformação: {ev_original:.6f} mm/mm")
                print(f" - MR corrigido: {revest_E_corrigido:.2f} MPa | Deformação: {ev_corrigida:.6f} mm/mm")
                print(f" - Deformação confiável ({nivel_confianca.value:.0f}%): {ev_confiavel:.6f} mm/mm\n")

                C = calcular_C(ev_confiavel)
                N = calcular_N(dec.value, cres.value, vida_util.value)
                fx, pc = obter_pc_por_faixa(nome, h_total_cm, dados_aeronaves_completos, wander_std=wander_std_widget.value)
                if fx_ref is None:
                    fx_ref = fx
                cdf = N / (pc * C)
                cdf_total += cdf
                plt.plot(fx, cdf, label=f"{nome}")

        if fx_ref is None:
            fx_ref = faixas
        plt.plot(fx_ref, cdf_total, color='black', linestyle='--', linewidth=3, label='CDF Total (soma)')
        plt.axhline(1, color='red', linestyle='--', label='Limite CDF = 1')
        plt.title('Distribuição do CDF ao longo da largura da pista')
        plt.xlabel('Posição lateral na pista (cm)')
        plt.ylabel('CDF')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()

executar.on_click(rodar_plot)

# === DISPLAY FINAL ===
display(widgets.VBox([
    widgets.HTML("<h3>Características do Pavimento e Temperatura</h3>"),
    widgets.HBox([temp_anual, revest_E]),
    revest_h,
    widgets.HBox([base_E, base_h]),
    widgets.HBox([subbase_E, subbase_h]),
    subleito_E,
    widgets.HBox([vida_util, wander_std_widget]),
    wander_std_explica,
    cov_explica,
    confiab_section,
    widgets.HTML("<h3>Dados de Aeronaves (Decolagens e Crescimento)</h3>"),
    widgets.VBox([widgets.HBox([a[1], a[2]]) for a in aeronave_inputs]),
    executar,
    saida
]))


In [ ]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.2 MB/s eta 0:00:00


In [ ]:
# streamlit_app.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
import itertools

# === CARREGAR MODELO ===
modelo = joblib.load('modelo_4_camadas_ESRS_ESRD_COMPLETO.pkl')
weights = modelo['weights']
scaler = modelo['scaler']
aeronaves_dict = modelo['aeronaves']

# === Funções ===
def tanh(x): return np.tanh(x)
def neural_network(weights, X):
    W1 = weights[:9 * 10].reshape((9, 10))
    b1 = weights[90:100]
    W2 = weights[100:180].reshape((10, 8))
    b2 = weights[180:188]
    W3 = weights[-9:-1].reshape((8, 1))
    b3 = weights[-1]
    hidden1 = tanh(np.dot(X, W1) + b1)
    hidden2 = tanh(np.dot(hidden1, W2) + b2)
    output = np.dot(hidden2, W3) + b3
    return output.flatten()

def prever_deformacao_customizada(valores):
    entrada = np.array([valores])
    entrada_norm = scaler.transform(entrada)
    return neural_network(weights, entrada_norm)[0]

def corrigir_mr_para_Tc(E25_mpa, Tc):
    return E25_mpa * np.exp(-0.08 * (Tc - 25))

def calcular_confiabilidade_rosenblueth(media_inputs, covs, confianca):
    indices = {
        'revest_MR': 2, 'revest_h': 3,
        'base_MR': 4, 'base_h': 5,
        'subbase_MR': 6, 'subbase_h': 7,
        'subleito_MR': 8
    }
    variaveis = list(indices.items())
    mu_vector = media_inputs.copy()
    cov_values = [covs[k] / 100 for k, _ in variaveis]

    if all(c == 0 for c in cov_values):
        st.warning("Todos os COVs estão zerados. A análise será determinística.")

    combinacoes = list(itertools.product([-1, 1], repeat=7))
    resultados = []
    for sinais in combinacoes:
        v = mu_vector.copy()
        for i, (_, idx) in enumerate(variaveis):
            v[idx] = mu_vector[idx] * (1 + sinais[i] * cov_values[i])
        f = prever_deformacao_customizada(v)
        resultados.append(f)

    media = np.mean(resultados)
    desvio = np.std(resultados)
    z = stats.norm.ppf(confianca / 100)
    confiavel = media + z * desvio
    return media, confiavel

def calcular_C(ev):
    if ev >= 1.765e-3:
        return (0.00414131 / ev) ** 8.1
    else:
        log_C = (-0.1638 + 185.19 * ev) ** (-0.60586)
        return 10 ** log_C

def calcular_N(a, b, L):
    return (1 + (b * L / 200)) * a * L

def calcular_cp(faixa_centro, weq, pos_pneus, wander_std):
    return sum([
        (stats.norm.cdf(faixa_centro + weq/2, pneu_pos, wander_std) -
         stats.norm.cdf(faixa_centro - weq/2, pneu_pos, wander_std))
        for pneu_pos in pos_pneus
    ])

def obter_pc_por_faixa(aeronave, h_total, wander_std):
    faixa_largura = 25.4
    faixas = np.linspace(-40 * faixa_largura, 40 * faixa_largura, 81)
    if aeronave['Ne'] == 1 or h_total >= aeronave['t'] - aeronave['w']:
        weq = aeronave['w'] + aeronave['t'] + h_total if aeronave['Ne'] > 1 else aeronave['w'] + h_total
        posicoes = [aeronave['xk_centro']]
    else:
        weq = aeronave['w'] + h_total
        desloc = aeronave['t'] / 2
        posicoes = [aeronave['xk_centro'] - desloc, aeronave['xk_centro'] + desloc]
    posicoes += [-p for p in posicoes]
    cp_vals = np.array([calcular_cp(f, weq, posicoes, wander_std) for f in faixas])
    return faixas, 1 / cp_vals

# === INTERFACE STREAMLIT ===
st.title("Predição de Deformações em Pavimentos Aeroportuários com RNA")

st.header("Dados do Pavimento")
Tc = st.number_input("Temperatura média anual (°C)", value=27.0)
revest_MR = st.number_input("MR 25°C Revestimento (MPa)", value=2500.0)
revest_h = st.number_input("Espessura do Revestimento (m)", value=0.3)
base_MR = st.number_input("MR Base (MPa)", value=300.0)
base_h = st.number_input("Espessura da Base (m)", value=0.3)
subbase_MR = st.number_input("MR Subbase (MPa)", value=150.0)
subbase_h = st.number_input("Espessura da Subbase (m)", value=0.4)
subleito_MR = st.number_input("MR Subleito (MPa)", value=60.0)
vida_util = st.number_input("Vida útil (anos)", value=20.0)
wander_std = st.number_input("Wander std (cm)", value=77.3)

st.markdown("---")
st.header("Configurações de Confiabilidade")
cov_padrao = {
    'revest_MR': 15, 'base_MR': 20, 'subbase_MR': 20, 'subleito_MR': 20,
    'revest_h': 7, 'base_h': 12, 'subbase_h': 15
}
covs = {}
for key, val in cov_padrao.items():
    covs[key] = st.number_input(f'COV {key} (%)', value=float(val))
confianca = st.number_input("Nível de Confiabilidade (%)", value=95.0)

st.markdown("---")
st.header("Dados das Aeronaves")
cdf_total = np.zeros(81)
faixas_ref = np.linspace(-40 * 25.4, 40 * 25.4, 81)

for nome, props in aeronaves_dict.items():
    col1, col2 = st.columns(2)
    with col1:
        a = st.number_input(f"Decolagens anuais {nome}", key=f"{nome}_a", value=0)
    with col2:
        b = st.number_input(f"Crescimento b (%) {nome}", key=f"{nome}_b", value=0.0)

    if a > 0:
        entrada = [
            props['Pressão dos Pneus(MPa)'],
            props['dist_rodas(m)'],
            revest_MR, revest_h,
            base_MR, base_h,
            subbase_MR, subbase_h,
            subleito_MR
        ]
        entrada_corrigida = entrada.copy()
        entrada_corrigida[2] = corrigir_mr_para_Tc(revest_MR, Tc)
        ev_med, ev_conf = calcular_confiabilidade_rosenblueth(entrada_corrigida, covs, confianca)
        C = calcular_C(ev_conf)
        N = calcular_N(a, b, vida_util)
        faixas, pc = obter_pc_por_faixa(props, (revest_h + base_h + subbase_h) * 100, wander_std)
        cdf = N / (pc * C)
        cdf_total += cdf
        st.success(f"{nome} - Deformação confiável: {ev_conf:.6e} mm/mm")

st.markdown("---")
st.subheader("Distribuição de Dano Acumulado (CDF)")
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(faixas_ref, cdf_total, label="CDF Total", color="black", linewidth=2)
ax.axhline(1, color='red', linestyle='--', label="Limite CDF = 1")
ax.set_xlabel("Posição lateral (cm)")
ax.set_ylabel("CDF")
ax.grid(True)
ax.legend()
st.pyplot(fig)


2025-04-16 00:26:15.046 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-16 00:26:15.047 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-16 00:26:15.049 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-16 00:26:15.052 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-16 00:26:15.054 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-16 00:26:15.056 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-16 00:26:15.058 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-16 00:26:15.059 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()